## pyfixest examples

Now supported: 

- multiple estimations via `sw()`, `csw()`, `sw0()` and `csw0()`

Currently not (yet) supported: 

- any type of missing values in the covariates, dependent variables, cluster variables (missings in fixed effects should be fine)
- also - is inference correct? minor deviations to `fixest`, maybe only small sample corrections?

In [1]:
# create some data

import pandas as pd
import numpy as np
from pyfixest.api import feols
from pyfixest.utils import get_data

In [2]:
data = get_data()

In [3]:
# HC1 inference
feols('Y ~ X1 | X2 + X3 + X4', 'HC3', data)

,fml,coefnames,coef,se,tstat,pvalue
0,Y~X1|X2+X3+X4,X1,-0.010369,0.010071,-1.0296,0.303198


In [5]:
# HC3 inference
feols('Y ~ X1 | X2 + X3 + X4', vcov = {'CRV1':'group_id'}, data = data)

,fml,coefnames,coef,se,tstat,pvalue
0,Y~X1|X2+X3+X4,X1,-0.010369,0.009333,-1.111012,0.266563


In [6]:
# multiple estimation: multiple dependent variables
feols('Y + Y2 ~ X1 | X2 + X3 + X4', 'HC3', data)

,fml,coefnames,coef,se,tstat,pvalue
0,Y~X1|X2+X3+X4,X1,-0.010369,0.010071,-1.029600,0.303198
0,Y2~X1|X2+X3+X4,X1,-0.027158,0.014045,-1.933662,0.053155


In [7]:
# sw 
# multiple estimation: multiple dependent variables
feols('Y + Y2 ~ X1 | sw0(X2, X3, X4)', 'HC3', data)

,fml,coefnames,coef,se,tstat,pvalue
0,Y~X1|0,X1,-0.157581,0.202136,-0.779580,0.435638
0,Y2~X1|0,X1,-0.173611,0.202379,-0.857850,0.390975
0,Y~X1|X2,X1,-0.103285,0.172931,-0.597259,0.550334
0,Y2~X1|X2,X1,-0.120207,0.173237,-0.693891,0.487750
0,Y~X1|X3,X1,-0.117885,0.178623,-0.659961,0.509279
0,Y2~X1|X3,X1,-0.134384,0.178959,-0.750921,0.452700
0,Y~X1|X4,X1,-0.135019,0.092803,-1.454907,0.145695
0,Y2~X1|X4,X1,-0.150129,0.093430,-1.606854,0.108086


In [8]:
# sw 
# multiple estimation: multiple dependent variables
feols(fml = 'Y + Y2 ~ sw(X1, X2) | csw0(X3, X4)', vcov = {'CRV1':'group_id'}, data = data)

,fml,coefnames,coef,se,tstat,pvalue
0,Y~X1|0,X1,-0.157581,0.198465,-0.794000,0.427195
0,Y~X2|0,X2,0.091878,0.004974,18.473397,0.000000
0,Y2~X1|0,X1,-0.173611,0.197312,-0.879882,0.378923
0,Y2~X2|0,X2,0.091482,0.005035,18.170610,0.000000
0,Y~X1|X3,X1,-0.117885,0.154998,-0.760557,0.446922
0,Y~X2|X3,X2,-0.515981,0.009493,-54.354837,0.000000
0,Y2~X1|X3,X1,-0.134384,0.153802,-0.873746,0.382257
0,Y2~X2|X3,X2,-0.516327,0.009654,-53.485357,0.000000
0,Y~X1|X3+X4,X1,-0.063646,0.082916,-0.767600,0.442725
0,Y~X2|X3+X4,X2,-0.509159,0.000583,-872.642351,0.000000
